# Processing the results of assited labeling

The assited labeling notebook yields several csv files one for each query. As the process have been parallelized, we have as many copies as threads we have run. So there is the need for merging the results in one single excel file per policy instrument.

In [31]:
import boto3
import codecs
import csv
import json
import numpy as np
import pandas as pd
import xlsxwriter

## Connecting to S3

In [3]:
path = "C:/Users/jordi/Documents/claus/"
path = "/home/propietari/Documents/claus/"
filename = "AWS_S3_keys_wri.json"
aws_id, aws_secret = aws_credentials(path, filename)
region = 'us-east-1'

bucket = 'wri-nlp-policy'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = aws_id,
    aws_secret_access_key = aws_secret
)

## Functions

In [52]:
def aws_credentials(path, filename):
    file = path + filename
    with open(file, 'r') as dict:
        key_dict = json.load(dict)
    for key in key_dict:
        KEY = key
        SECRET = key_dict[key]
    return KEY, SECRET

def read_csv_from_s3(s3_object, columns):
    pre_labeled = []
    for i, row in enumerate(csv.DictReader(codecs.getreader("utf-8")(s3_object.get()['Body']))):
        list_row = []
        for column in columns:
            list_row.append(row[column])
        list_row.insert(0, i)
        pre_labeled.append(list_row)
    return pre_labeled

def export_list_as_excel(file, list_to_save, name_of_sheet):
    df = pd.DataFrame(list_to_save)
    writer = pd.ExcelWriter(file, engine='openpyxl', mode = "a")
    if name_of_sheet not in writer.sheets:
        writer.sheets[name_of_sheet] = 0
    df.to_excel(writer, sheet_name = name_of_sheet, startrow=writer.sheets[name_of_sheet].max_row, index=False, header=False)
    writer.save()

## Loading the policy instruments

In [20]:
# Define queries
path = "../../input/"
filename = "English_queries.xlsx"
file = path + filename
df = pd.read_excel(file, engine='openpyxl', sheet_name = "Hoja1", usecols = "A:C")

policy_intrument = {}
for index, row in df.iterrows():
    policy_intrument[row['Policy instrument']] = 0

## Loading csv files

In [53]:
path = "../../output/"
filename = "pre_labeled_English.xlsx"
file = path + filename

columns = ["sentence_id", "similarity_score", "text"]
for i, obj in enumerate(s3.Bucket(bucket).objects.all().filter(Prefix="english_documents/assisted_labeling/")):
    if not obj.key.endswith("/") and i < 65:
        for item in policy_intrument:
            if item in obj.key:
                export_list_as_excel(file, read_csv_from_s3(obj, columns), item)
#         print(read_csv_from_s3(obj, columns))

AttributeError: 'int' object has no attribute 'max_row'